In [32]:
import sys
import os
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from spatial_graphs.Stats import Stats
import pathlib

In [9]:
input_path = '/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/'

for file_name in ['AN9_20201103_v2_left.am']:#['AN5_20201103_v4_left.am','AN6_20201103_v8_left_new.am',]:

    sg = AmiraSpatialGraph(input_path+file_name)
    sg_new = AmiraSpatialGraph()
    sg_new.graph_data = sg.pia
    sg_new.graph_data.set_label('9')
    sg_new.write_spatial_graph(input_path+file_name[:-3]+'_relabelled.am')

    # Generate surface
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
                  format(input_path+file_name[:-3]+'_relabelled.am',\
                         input_path+file_name[:-3]+'_relabelled',str(100)))

In [35]:
# First change the view of the input data so that xy view is always top view
tr_mat = [1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 1]
input_path = '/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/To_Be_Transformed/'
for folder in ['AN5_20201103_v4_left','AN6_20201103_v8_left','AN9_20201103_v2_left']:
    output_path = input_path+folder+'/Canonical/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    aligner.transform_folder(input_path+folder+'/',output_path,txmat = tr_mat,apply_polydata_transform=True)

In [54]:
# register surf to mg 48
rf_surf_path_lhs = '/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs/Surfaces/MG48_lhs_pia_voxel_size_100.vtk'
rf_surf_lhs = Surface(rf_surf_path_lhs)
rf_center = rf_surf_lhs.surface.GetCenter()
output_path = '/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/' + 'Registered_To_MG48/LHS-LHS/'

input_path = '/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/To_Be_Transformed/'
for folder in ['AN5_20201103_v4_left','AN6_20201103_v8_left','AN9_20201103_v2_left']:
    pia = Surface(input_path+folder+'/Canonical/'+folder+'_relabelled_pia.vtk')
    
    aligner = Alignment(rf_surf_lhs.surface,pia.surface,nr_iterations=10000)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    aligner.transform_folder(input_path+folder+'/Canonical/',output_path,icp=icp,txmat = txmat,\
                            txfilename = output_path+folder+'_Transformation_Matrix.txt')
    